In [5]:
"""
1.Estimate intensity of atmospheric light
2.Estimate transmission map
t(x) = 1 – defoggingParam * darkPixelFromCoarseMap / AtmosphericLightIntensity
3.Clarify image
J(x)=(I(x)- A)/max(t(x), t0)+A

I(x) = J(x)t(x) + A(1 - t(x))
where I is the observed image intensity,
J is the scene radiance, 
A is the global atmospheric light and 
t is the medium transmission,



"""



from __future__ import division
import cv2
import os
import numpy as np


class Channel_value:
    val = -1.0
    intensity = -1.0
#Find the top 0.1% brightest pixels in the dark channel
#then choose one with highest intensity as the representing of atmospheric light.
#Finding the pixel with the highest atmospheric light
## for example img.shape return (1620, 2729, 3)
def atmospheric_light(img, gray):
    top_num = int(img.shape[0] * img.shape[1] * 0.001)  #output 4420
    toplist = [Channel_value()] * top_num
    dark_channel = dark_channel_find(img)
#loop through each value 
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            #Accessing all RGB values using img.item
            val = img.item(y, x, dark_channel)
            intensity = gray.item(y, x)
            for t in toplist:
                if t.val < val or (t.val == val and t.intensity < intensity):
                    t.val = val
                    t.intensity = intensity
                    break
    max_channel = Channel_value()
    for t in toplist:
        if t.intensity > max_channel.intensity:
            max_channel = t
    return max_channel.intensity

#Finding the dark channel i.e. the pixel with the lowest R/G/B value
def dark_channel_find(img):
    return np.unravel_index(np.argmin(img), img.shape)[2]

#Finding a coarse image which gives us a transmission map
def coarse(minimum, x, maximum):
    return max(minimum, min(x, maximum))

#Uses values from other functions to aggregate and give us a clear image
def dehaze(img, light_intensity, windowSize, t0, w):
    size = (img.shape[0], img.shape[1])
#Return a new array of given shape and type, filled with zeros.
    outimg = np.zeros(img.shape, img.dtype)

    for y in range(size[0]):
        for x in range(size[1]):
            x_low = max(x-(windowSize//2), 0)
            y_low = max(y-(windowSize//2), 0)
            x_high = min(x+(windowSize//2), size[1])
            y_high = min(y+(windowSize//2), size[0])

            sliceimg = img[y_low:y_high, x_low:x_high]

            dark_channel = dark_channel_find(sliceimg)
            #Estimate Transmission map
            t = 1.0 - (w * img.item(y, x, dark_channel) / light_intensity)

            outimg.itemset((y,x,0), coarse(0, ((img.item(y,x,0) - light_intensity) / max(t, t0) + light_intensity), 255))
            outimg.itemset((y,x,1), coarse(0, ((img.item(y,x,1) - light_intensity) / max(t, t0) + light_intensity), 255))
            outimg.itemset((y,x,2), coarse(0, ((img.item(y,x,2) - light_intensity) / max(t, t0) + light_intensity), 255))
    return outimg


def main():
    d="C:\\Users\\Uma\\Anacondanew\\envs\\uma_tensor\\haze_removal\\Input_Images\\"
    cd="C:\\Users\\Uma\\Anacondanew\\envs\\uma_tensor\\haze_removal\\out_img\\out\\"

    
    files=os.listdir(d)
    for f in files:
        img = cv2.imread(d+f)
        img = np.array(img, dtype=np.uint8)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        light_intensity = atmospheric_light(img, gray)
        w = 0.95
        t0 = 0.55
        outimg = dehaze(img, light_intensity, 20, t0, w)
        name = cd+f
            #print( name)
        cv2.imwrite(name, outimg)

if __name__ == "__main__": main()